In [ ]:
#327, 281, 270 w/ nuclei annotations

In [13]:
import tifffile
import cv2, tqdm
import numpy as np
import matplotlib, matplotlib.pyplot as plt
%matplotlib inline
matplotlib.rcParams['figure.dpi']=300

import pandas as pd
import torch 
from sklearn.model_selection import train_test_split
import glob, os, pickle
from torch.utils.data import DataLoader
from torch import nn
from matplotlib import cm

In [69]:
base_dir='/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/projects/scc_nuclei/data/nuclei_contour_annotations/'
xml_files=['270_A1d_ASAP.xml', '281_A1f_ASAP.xml', '327_A1d_ASAP.xml']
img_files=["270_A1d_ASAP.tif", "281_A1f_ASAP.tif", "327_A1d_ASAP.tif"]
annotations={}

In [33]:
from pathpretrain.process_xml import process_xml

In [70]:
for f in tqdm(xml_files): #rerun this to reset the coordinates after transformation 
    annotations[f] = process_xml(f'/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/projects/scc_nuclei/data/nuclei_contour_annotations/{f}', transpose = False)['contour']

  0%|                                                            | 0/3 [00:00<?, ?it/s]/dartfs-hpc/rc/home/3/f006n33/anaconda3/envs/hiss/lib/python3.8/site-packages/scipy/interpolate/_fitpack_impl.py:226: RuntimeWarning: Setting x[0][33]=x[0][0]
  warnings.warn(RuntimeWarning('Setting x[%d][%d]=x[%d][0]' %
/dartfs-hpc/rc/home/3/f006n33/anaconda3/envs/hiss/lib/python3.8/site-packages/scipy/interpolate/_fitpack_impl.py:226: RuntimeWarning: Setting x[1][33]=x[1][0]
  warnings.warn(RuntimeWarning('Setting x[%d][%d]=x[%d][0]' %
/dartfs-hpc/rc/home/3/f006n33/anaconda3/envs/hiss/lib/python3.8/site-packages/scipy/interpolate/_fitpack_impl.py:226: RuntimeWarning: Setting x[0][261]=x[0][0]
  warnings.warn(RuntimeWarning('Setting x[%d][%d]=x[%d][0]' %
/dartfs-hpc/rc/home/3/f006n33/anaconda3/envs/hiss/lib/python3.8/site-packages/scipy/interpolate/_fitpack_impl.py:226: RuntimeWarning: Setting x[1][261]=x[1][0]
  warnings.warn(RuntimeWarning('Setting x[%d][%d]=x[%d][0]' %
/dartfs-hpc/rc/home/3/f006n

 33%|█████████████████▎                                  | 1/3 [00:00<00:01,  1.82it/s]/dartfs-hpc/rc/home/3/f006n33/anaconda3/envs/hiss/lib/python3.8/site-packages/scipy/interpolate/_fitpack_impl.py:226: RuntimeWarning: Setting x[0][6]=x[0][0]
  warnings.warn(RuntimeWarning('Setting x[%d][%d]=x[%d][0]' %
/dartfs-hpc/rc/home/3/f006n33/anaconda3/envs/hiss/lib/python3.8/site-packages/scipy/interpolate/_fitpack_impl.py:226: RuntimeWarning: Setting x[1][6]=x[1][0]
  warnings.warn(RuntimeWarning('Setting x[%d][%d]=x[%d][0]' %
/dartfs-hpc/rc/home/3/f006n33/anaconda3/envs/hiss/lib/python3.8/site-packages/scipy/interpolate/_fitpack_impl.py:226: RuntimeWarning: Setting x[0][4]=x[0][0]
  warnings.warn(RuntimeWarning('Setting x[%d][%d]=x[%d][0]' %
/dartfs-hpc/rc/home/3/f006n33/anaconda3/envs/hiss/lib/python3.8/site-packages/scipy/interpolate/_fitpack_impl.py:226: RuntimeWarning: Setting x[1][4]=x[1][0]
  warnings.warn(RuntimeWarning('Setting x[%d][%d]=x[%d][0]' %
/dartfs-hpc/rc/home/3/f006n33/ana

/dartfs-hpc/rc/home/3/f006n33/anaconda3/envs/hiss/lib/python3.8/site-packages/scipy/interpolate/_fitpack_impl.py:226: RuntimeWarning: Setting x[0][12]=x[0][0]
  warnings.warn(RuntimeWarning('Setting x[%d][%d]=x[%d][0]' %
/dartfs-hpc/rc/home/3/f006n33/anaconda3/envs/hiss/lib/python3.8/site-packages/scipy/interpolate/_fitpack_impl.py:226: RuntimeWarning: Setting x[1][12]=x[1][0]
  warnings.warn(RuntimeWarning('Setting x[%d][%d]=x[%d][0]' %
/dartfs-hpc/rc/home/3/f006n33/anaconda3/envs/hiss/lib/python3.8/site-packages/scipy/interpolate/_fitpack_impl.py:226: RuntimeWarning: Setting x[0][13]=x[0][0]
  warnings.warn(RuntimeWarning('Setting x[%d][%d]=x[%d][0]' %
/dartfs-hpc/rc/home/3/f006n33/anaconda3/envs/hiss/lib/python3.8/site-packages/scipy/interpolate/_fitpack_impl.py:226: RuntimeWarning: Setting x[1][13]=x[1][0]
  warnings.warn(RuntimeWarning('Setting x[%d][%d]=x[%d][0]' %
100%|████████████████████████████████████████████████████| 3/3 [00:01<00:00,  1.97it/s]


In [113]:
imgs={}
Image.MAX_IMAGE_PIXELS = None
for i in tqdm(img_files):
    imgs[i]=np.asarray(Image.open(f'/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/projects/scc_nuclei/data/nuclei_contour_annotations/{i}'))

100%|█████████████████████████████████████████████████| 3/3 [00:56<00:00, 18.67s/it]


In [39]:
annotations

{'270_A1d':                                                contours         xmin  \
 0     [[12060.473600000001, 14175.118200000004], [12...   8649.70898   
 1     [[49675.234400000016, 21421.240200000004], [49...  46002.21480   
 2     [[48522.8047, 21085.607400000008], [48518.7028...  47249.69920   
 3     [[49110.0781, 20878.914100000013], [49109.0973...  48937.95310   
 4     [[47403.84380000001, 25147.882799999996], [474...  47219.22270   
 ...                                                 ...          ...   
 1133  [[11497.503899999996, 18522.990199999997], [11...  11497.50390   
 1134  [[11350.170900000001, 18447.0195], [11350.1648...  11350.17090   
 1135  [[11378.2559, 18446.099599999998], [11378.2782...  11365.82420   
 1136  [[11081.539099999998, 18519.509800000003], [11...  11081.53910   
 1137  [[12979.521499999999, 18529.828100000006], [12...  12957.36430   
 
             xmax        ymin        ymax         xmean         ymean  \
 0     16591.8633  14175.1182  25381.8

In [71]:
print(annotations["327_A1d_ASAP.xml"]["lbl"].unique())

['scc nuclei' 'inflammatory cells']


In [72]:
annotations['327_A1d_ASAP.xml']

,contours,xmin,xmax,ymin,ymax,xmean,ymean,lbl
0,"[[19183.445299999992, 18803.369099999996], [19...",19166.5527,19183.4453,18793.3867,18809.1270,19174.176057,18801.394800,scc nuclei
1,"[[19184.595699999998, 18784.173799999997], [19...",19166.9355,19191.5059,18781.1016,18797.2266,19179.861000,18790.380217,scc nuclei
2,"[[19300.539100000005, 18805.287100000005], [19...",19299.7715,19320.1172,18792.2363,18811.4316,19308.107143,18802.546029,scc nuclei
3,"[[19326.644500000002, 18813.3496], [19326.6964...",19315.1270,19331.6348,18810.6641,18830.2422,19323.402578,18820.730044,scc nuclei
4,"[[19335.089799999994, 18801.0664], [19335.0719...",19335.0898,19348.5273,18796.0742,18812.5840,19340.904000,18804.027343,scc nuclei
...,...,...,...,...,...,...,...,...
1349,"[[18192.4355, 20206.636699999992], [18192.3949...",18189.3574,18206.6660,20206.6367,20228.5605,18197.947900,20217.662750,scc nuclei
1350,"[[18206.281300000002, 20191.2539], [18206.3233...",18192.4355,18207.0508,20188.5605,20208.1758,18200.319983,20198.304350,scc nuclei
1351,"[[18122.4355, 20192.021499999995], [18122.4001...",18115.1289,18127.8203,20192.0215,20208.1758,18121.282233,20200.418967,scc nuclei
1352,"[[17972.4375, 20183.945300000003], [17972.4779...",17957.0547,17974.7461,20179.7148,20196.2520,17966.174671,20188.176071,scc nuclei


In [125]:
def patchify(sample_id, img_parent_dir, df_parent_dir, patch_data_parent_dir, img, annotations, img_size = 256):
    '''
    returns: image patches, along side metadata for each patch 
    '''
    general_df = pd.DataFrame() #general df will tell us about each patch in the WSI 
    '''
    More specifically, detectron patches will be a list[dict], where each dictionary stores the info stated above for the patch and it's correspoinding annotations
    '''
    detection_patches = [] #this is for detectron. This will tell us exactly what is in each patch. 
    
    #for general metadata df 
    xs = []
    ys = []
    sizes = []
    patch_ids = []
    filepaths = []
    inflamm = [] #1 is true 
    scc = [] # 1 is true 
    
    #corner coords 
    x = 0 
    y = 0 
    
    #index
    idx = 0 
    for y in tqdm(range(0, img.shape[0], img_size)): #iterate through each potential patch
        for x in range(0, img.shape[1], img_size):
        
            
            img_patch = img[y:y+img_size, x:x+img_size] #get the img patch with upper-left corner at x,y 
            #find all the annotations within the bounds of this patch 
            #corner coords 
            x_min = x
            x_max = x + img_size
            y_min = y 
            y_max = y + img_size
            corner = (x_min, y_min)
            
            contours = [] #store list of outlines here
            
            patch_inflamm = False #track whether either is present at the patch level, which means that, at minimum, there is one annotation containing either
            patch_scc = False 
            
            #get all the contours that are within the patch, and assess patch level info
            for c, xmin, xmax, ymin, ymax, label in zip(annotations["contours"], annotations["xmin"], annotations["xmax"], annotations["ymin"], annotations["ymax"], annotations["lbl"]):
                if x_min < xmin < xmax <= x_max and y_min < ymin < ymax <= y_max:
                    #check to make sure that this is a nuclei contour by checking the size 
                    if (xmax - xmin) < 50 and (ymax - ymin) < 50: 
                        # c is just one annotation around one instance in the patch
                        contours.append(c)
                        if label == "inflammatory cells" or label == "inflammatory nuclei":
                            patch_inflamm = True
                        if label == "scc nuclei":
                            patch_scc = True 
                    
            #at this point, we know whether to further process this patch for general
            if len(contours) > 0:
                #get all the general df information
                xs.append(x)
                ys.append(y)
                patch_ids.append(idx)
                scc.append(int(patch_scc))
                inflamm.append(int(patch_inflamm))
                
                #SAVE THE IMAGE PATCH (this might be needed later, for panoptic segmentation)
                path =  str(img_parent_dir + "/" + sample_id + "_"+ str(idx)+".jpg")
                filepaths.append(path)
                
                patch = cv2.cvtColor(img_patch, cv2.COLOR_BGR2RGB)
                cv2.imwrite(path, patch)
#                 np.save(path, img_patch) #detectron wants this saved as a PIL 
#                 img_patch = im.fromarray(img_patch)
#                 img_patch.save(path)
                

                
                #now we process each specific object annotation in annotations for the patch
                for c, xmin, xmax, ymin, ymax, label in zip(annotations["contours"], annotations["xmin"], annotations["xmax"], annotations["ymin"], annotations["ymax"], annotations["lbl"]):
                    if x_min < xmin < xmax <= x_max and y_min < ymin < ymax <= y_max and (xmax - xmin) < 50 and (ymax - ymin) < 50: #ensure it's within bounds and a nuclei detection
                        #track whether scc/inflamm is in this specific object annotation
                        nuclei_scc = False
                        nuclei_inflamm = False
                        
                        if label == "scc nuclei":
                            nuclei_scc = True 
                        if label == "inflammatory cells" or label == "inflammatory nuclei":
                            nuclei_inflamm = True
                       
                        instance_annotation = {} #store in the coco format 
                        instance_annotation["bbox"] = [xmin - corner[0], ymin - corner[1], xmax - corner[0], ymax - corner[1]] # we need this to be contextualized relative to the corner coords
                        instance_annotation["bbox_mode"] = 0 # 0 = "BoxMode.XYXY_ABS"
                        #only two comparisons because the label is either SCC or inflamm or different
                        if nuclei_scc:
                            instance_annotation["real_id"] = 1
                        elif nuclei_inflamm:
                            instance_annotation["real_id"] = 0
                        else:
                            continue 
                            
                        instance_annotation["corner"] = corner
                        instance_annotation["category_id"] = 0 #make all of the cells 0 just for segmentation purposes
                        
                       
                        #nuclei patch thing?
                        patch_thing=img[int(instance_annotation['bbox'][0]):int(instance_annotation['bbox'][2]), int(instance_annotation['bbox'][1]):int(instance_annotation['bbox'][3])]
                        
                        #SAVE THE NUCLEI PATCH 
                        
                        nuclei_path =  str(img_parent_dir + "/" + sample_id + "_"+ str(idx)+"nuclei.jpg")
                        filepaths.append(nuclei_path)

                        nuclei_patch = cv2.cvtColor(patch_thing, cv2.COLOR_BGR2RGB)
                        cv2.imwrite(path, nuclei_patch)
                               


            
                detection_patches.append(instance_annotation)
                #update the patch index
                idx += 1
                
    #make the general df
    general_df["x"] = xs
    general_df["y"] = ys
    general_df["size"] = [img_size] * len(xs)
    general_df["patch_idx"] = patch_ids
    general_df["scc"] = scc
    general_df["inflamm"] = inflamm
    general_df["sample_id"] = [sample_id] * len(xs)
    #save df, which corresponds to that WSI 
    general_df.to_csv(df_parent_dir + "/" + sample_id + "_metadata.csv")
    
    #save the detectron patche data to the directory
    filepath = str(patch_data_parent_dir + "/" + sample_id + ".pickle")
    with open(filepath, 'wb') as handle:
        pickle.dump(detection_patches, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
    return detection_patches


In [126]:
img_parent_dir = "/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Sophie_Chen/nuclei_images"
df_parent_dir = "/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Sophie_Chen/nuclei_metadata"
detectron_data_parent_dir = "/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Sophie_Chen"

for e in tqdm(xml_files): 
    sample_id = e.split(".")[0]
    img_id = str(sample_id+".tif")
    WSI = imgs[img_id]
    detection_patches = patchify(sample_id, img_parent_dir, df_parent_dir, detectron_data_parent_dir, WSI, annotations[e])

 49%|██████████████████████▋                       | 91/185 [00:19<00:12,  7.27it/s]


 99%|████████████████████████████████████████████▊| 184/185 [00:31<00:00,  7.70it/s]


 76%|██████████████████████████████████▊           | 90/119 [00:19<00:03,  7.58it/s]


 36%|████████████████▋                             | 62/171 [00:08<00:14,  7.43it/s]


 91%|████████████████████████████████████████▊    | 155/171 [00:28<00:02,  7.51it/s]


100%|█████████████████████████████████████████████████| 3/3 [01:26<00:00, 28.68s/it]


In [78]:
import csv
from PIL import Image

def resize_images(folder_path, new_size):
    # create a list to store image metadata
    image_metadata = []
    for filename in os.listdir(folder_path):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            # open the image
            image = Image.open(os.path.join(folder_path, filename))
            # get the image label from the filename
            label = filename.split('_')[0]
            # resize the image
            image = image.resize((new_size, new_size))
            # save the image with the same name
           # image.save(os.path.join(folder_path, filename))
            # add image metadata to the list
            image_metadata.append([filename, label, new_size])
    return image_metadata

def create_metadata_csv(image_metadata, csv_file):
    # write the metadata to a csv file
    with open(csv_file, 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(['filepath', 'label', 'size'])
        csvwriter.writerows(image_metadata)

# define the folder path and new image size
folder_path = '/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Sophie_Chen/nuclei_images/'
new_size = 32
# resize images and create the metadata
image_metadata = resize_images(folder_path, new_size)
create_metadata_csv(image_metadata, 'metadata.csv')


In [84]:
ugh="/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Sophie_Chen/nuclei_metadata/270_A1d_ASAP.pickle"

In [85]:
df=pd.read_pickle(ugh)

In [102]:
df[0]['annotations']

[{'bbox': [164.28709999999955,
   205.52339999999822,
   181.77340000000004,
   222.3104999999996],
  'bbox_mode': 0,
  'real_id': 1,
  'segmentation': [[181.77340000000004,
    205.5234000000055,
    181.73308274444025,
    205.50400379475104,
    181.69224789793589,
    205.4850987816826,
    181.6509012321767,
    205.4666812000978,
    181.60904851887062,
    205.44874728932336,
    181.56669552970743,
    205.43129328866053,
    181.52384803637688,
    205.41431543741055,
    181.480511810596,
    205.39780997491107,
    181.43669262404546,
    205.3817731404488,
    181.39239624842958,
    205.36620117335042,
    181.34762845544356,
    205.35109031291722,
    181.30239501677534,
    205.33643679845773,
    181.25670170414196,
    205.32223686929865,
    181.21055428922045,
    205.30848676474125,
    181.16395854372422,
    205.29518272409405,
    181.11692023933938,
    205.28232098667286,
    181.06944514776478,
    205.26989779178257,
    181.0215390407011,
    205.2579093787